In [25]:
# ============================================
# 1. INSTALL DEPENDENCIES
# ============================================

print("Installing dependencies...")
!pip install -q -U pip
!pip install -q -U "transformers>=4.31.0" "huggingface_hub>=0.18.0"
!pip install -q git+https://github.com/csebuetnlp/normalizer
!pip install -q evaluate seqeval datasets accelerate


Installing dependencies...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
# ============================================
# 2. IMPORTS
# ============================================

print("\nImporting libraries...")
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
import evaluate
import numpy as np



Importing libraries...


In [27]:
# ============================================
# 3. GPU CHECK
# ============================================

print("\n" + "="*60)
print("GPU DIAGNOSTIC CHECK")
print("="*60)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda if torch.cuda.is_available() else 'N/A'}")

if torch.cuda.is_available():
    print(f"GPU count: {torch.cuda.device_count()}")
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    device = torch.device("cuda")
    print("✓ GPU is available and will be used!")
else:
    device = torch.device("cpu")
    print("⚠️ WARNING: No GPU detected! Training will be SLOW on CPU")
    print("In Colab: Runtime > Change runtime type > Hardware accelerator > GPU")



GPU DIAGNOSTIC CHECK
PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA version: 12.4
GPU count: 2
GPU name: Tesla T4
GPU memory: 15.83 GB
✓ GPU is available and will be used!


In [28]:
# ============================================
# 4. LOAD DATASET
# ============================================

print("\n" + "="*60)
print("LOADING WIKIANN BANGLA DATASET")
print("="*60)
dataset = load_dataset("wikiann", "bn")
print(dataset)
print(f"\nTrain size: {len(dataset['train'])}")
print(f"Validation size: {len(dataset['validation'])}")
print(f"Test size: {len(dataset['test'])}")

print("\nSample from training set:")
print(dataset["train"][0])

label_list = dataset["train"].features["ner_tags"].feature.names
print(f"\nNER Labels: {label_list}")

print("\n" + "-"*60)
print("SAMPLE SENTENCES WITH TAGS")
print("-"*60)
for i in range(3):
    tokens = dataset["train"][i]["tokens"]
    tags = [label_list[t] for t in dataset["train"][i]["ner_tags"]]
    print(f"\nSentence {i+1}:")
    print("Tokens:", " ".join(tokens))
    print("Tags:  ", " ".join(tags))



LOADING WIKIANN BANGLA DATASET
DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 1000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
})

Train size: 10000
Validation size: 1000
Test size: 1000

Sample from training set:
{'tokens': ['ড্যানভিল', ',', 'ইলিনয়'], 'ner_tags': [5, 6, 6], 'langs': ['bn', 'bn', 'bn'], 'spans': ['LOC: ড্যানভিল , ইলিনয়']}

NER Labels: ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

------------------------------------------------------------
SAMPLE SENTENCES WITH TAGS
------------------------------------------------------------

Sentence 1:
Tokens: ড্যানভিল , ইলিনয়
Tags:   B-LOC I-LOC I-LOC

Sentence 2:
Tokens: শিরোনামহীন শিরোনামহীন '' ( ২০১৩ )
Tags:   B-ORG B-ORG O O O O

Sentence 3:
Tokens: এই ইউনিয

In [29]:
# ============================================
# 5. LOAD TOKENIZER — INDICBERT
# ============================================

print("\n" + "="*60)
print("LOADING TOKENIZER (INDICBERT)")
print("="*60)
MODEL_NAME = "ai4bharat/IndicBERTv2-MLM-only"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(f"Tokenizer loaded: {MODEL_NAME}")



LOADING TOKENIZER (INDICBERT)
Tokenizer loaded: ai4bharat/IndicBERTv2-MLM-only


In [30]:
# ============================================
# 6. TOKENIZATION FUNCTION
# ============================================

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding=False,
        max_length=128
    )

    all_labels = []
    for i, labels in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(labels[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        all_labels.append(label_ids)

    tokenized_inputs["labels"] = all_labels
    return tokenized_inputs


In [31]:
# ============================================
# 7. TOKENIZE DATASET
# ============================================

print("\n" + "="*60)
print("TOKENIZING DATASET")
print("="*60)
tokenized_datasets = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    desc="Tokenizing"
)
tokenized_datasets = tokenized_datasets.remove_columns(["tokens", "ner_tags", "langs"])
print("Tokenized dataset:")
print(tokenized_datasets)



TOKENIZING DATASET


Tokenizing:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenized dataset:
DatasetDict({
    validation: Dataset({
        features: ['spans', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['spans', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    train: Dataset({
        features: ['spans', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
})


In [32]:
# ============================================
# 8. LOAD MODEL — INDICBERT
# ============================================

print("\n" + "="*60)
print("LOADING MODEL (INDICBERT)")
print("="*60)
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(label_list),
    id2label={i: label for i, label in enumerate(label_list)},
    label2id={label: i for i, label in enumerate(label_list)}
)
model = model.to(device)
print(f"Model loaded: {MODEL_NAME}")
print(f"Model device: {next(model.parameters()).device}")
print(f"Number of parameters: {model.num_parameters():,}")



LOADING MODEL (INDICBERT)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at ai4bharat/IndicBERTv2-MLM-only and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded: ai4bharat/IndicBERTv2-MLM-only
Model device: cuda:0
Number of parameters: 277,456,135


In [33]:
# ============================================
# 9. SETUP METRICS
# ============================================

print("\n" + "="*60)
print("LOADING METRICS")
print("="*60)
metric = evaluate.load("seqeval")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }



LOADING METRICS


In [34]:
# ============================================
# 10. DATA COLLATOR
# ============================================

print("\n" + "="*60)
print("SETTING UP DATA COLLATOR")
print("="*60)
data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer,
    padding=True,
    label_pad_token_id=-100
)
print("Data collator created successfully!")



SETTING UP DATA COLLATOR
Data collator created successfully!


In [35]:
# ============================================
# 11. TRAINING ARGUMENTS
# ============================================

print("\n" + "="*60)
print("SETTING UP TRAINING ARGUMENTS")
print("="*60)

training_args = TrainingArguments(
    output_dir="./results-banglabert-ner",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=torch.cuda.is_available(),
    push_to_hub=False,
    save_total_limit=2,
    report_to="none",
)

print(f"Training device: {training_args.device}")
print(f"FP16 training: {training_args.fp16}")
print(f"Batch size: {training_args.per_device_train_batch_size}")
print(f"Number of epochs: {training_args.num_train_epochs}")



SETTING UP TRAINING ARGUMENTS
Training device: cuda:0
FP16 training: True
Batch size: 16
Number of epochs: 5


In [36]:
# ============================================
# 12. CREATE TRAINER
# ============================================

print("\n" + "="*60)
print("CREATING TRAINER")
print("="*60)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print("Trainer created successfully!")



CREATING TRAINER
Trainer created successfully!


In [37]:
# ============================================
# 13. START TRAINING
# ============================================

print("\n" + "="*60)
print("STARTING TRAINING")
print("="*60)
print(f"This will take approximately 15–25 minutes on a T4 GPU")
print("="*60 + "\n")

trainer.train()

print("\n" + "="*60)
print("TRAINING COMPLETED!")
print("="*60)


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 3}.



STARTING TRAINING
This will take approximately 15–25 minutes on a T4 GPU



/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.210400,0.164587,0.911894,0.934959,0.923283,0.954598
2,0.095500,0.140970,0.936607,0.947606,0.942075,0.966521
3,0.089900,0.104335,0.957998,0.968383,0.963163,0.977299
4,0.028900,0.117683,0.957105,0.967480,0.962264,0.977069
5,0.025100,0.110342,0.964061,0.969286,0.966667,0.978675


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



TRAINING COMPLETED!


In [38]:
# ============================================
# 14. FINAL EVALUATION
# ============================================

print("\n" + "="*60)
print("FINAL EVALUATION ON VALIDATION SET")
print("="*60)
eval_results = trainer.evaluate()
print("\nValidation Results:")
for key, value in eval_results.items():
    print(f"  {key}: {value:.4f}")

print("\n" + "="*60)
print("EVALUATION ON TEST SET")
print("="*60)
test_results = trainer.evaluate(tokenized_datasets["test"])
print("\nTest Results:")
for key, value in test_results.items():
    print(f"  {key}: {value:.4f}")



FINAL EVALUATION ON VALIDATION SET


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



Validation Results:
  eval_loss: 0.1103
  eval_precision: 0.9641
  eval_recall: 0.9693
  eval_f1: 0.9667
  eval_accuracy: 0.9787
  eval_runtime: 4.0432
  eval_samples_per_second: 247.3280
  eval_steps_per_second: 7.9150
  epoch: 5.0000

EVALUATION ON TEST SET

Test Results:
  eval_loss: 0.1030
  eval_precision: 0.9661
  eval_recall: 0.9679
  eval_f1: 0.9670
  eval_accuracy: 0.9799
  eval_runtime: 12.4639
  eval_samples_per_second: 80.2320
  eval_steps_per_second: 2.5670
  epoch: 5.0000


In [39]:
# ============================================
# 15. SAVE MODEL
# ============================================

print("\n" + "="*60)
print("SAVING FINAL MODEL")
print("="*60)
trainer.save_model("./indicbert-ner-final")
tokenizer.save_pretrained("./indicbert-ner-final")
print("Model saved to: ./indicbert-ner-final")



SAVING FINAL MODEL
Model saved to: ./indicbert-ner-final


In [40]:
import shutil
import os

# Define the output zip file
output_filename = "kaggle_working_dir.zip"

# Current working directory
current_dir = os.getcwd()

# Create a zip of the current directory
shutil.make_archive("kaggle_working_dir", 'zip', current_dir)

print(f"✅ ZIP file created: {output_filename}")


✅ ZIP file created: kaggle_working_dir.zip


In [41]:
# ============================================
# 16. TEST INFERENCE
# ============================================

print("\n" + "="*60)
print("TESTING INFERENCE")
print("="*60)

test_sentence = dataset["test"][0]
test_tokens = test_sentence["tokens"]
print(f"\nTest sentence: {' '.join(test_tokens)}")

inputs = tokenizer(
    test_tokens,
    is_split_into_words=True,
    return_tensors="pt",
    truncation=True,
    padding=True
).to(device)

with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=2)

predicted_labels = [label_list[p.item()] for p in predictions[0]]
word_ids = inputs.word_ids()

final_predictions = []
previous_word_idx = None
for word_idx, pred_label in zip(word_ids, predicted_labels):
    if word_idx is not None and word_idx != previous_word_idx:
        final_predictions.append(pred_label)
        previous_word_idx = word_idx

print("\nPredicted NER tags:")
for token, pred_tag in zip(test_tokens, final_predictions):
    print(f"  {token:20s} -> {pred_tag}")

print("\n" + "="*60)
print("ALL DONE! 🎉")
print("="*60)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



TESTING INFERENCE

Test sentence: উরুগুয়ে জাতীয় ফুটবল দল

Predicted NER tags:
  উরুগুয়ে             -> B-ORG
  জাতীয়               -> I-ORG
  ফুটবল                -> I-ORG
  দল                   -> I-ORG

ALL DONE! 🎉
